# LUNA Lung Segmentation ROI


# Dependency Descriptions
1. **skimage.morphology**: Morphological operations, e.g., opening or skeletonization.
2. **skimage.measure**: Measurement of image properties, e.g., similarity and contours.
3. **sklearn.cluster.KMeans**: clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares
4. **skimage.transform.resize**: Geometric and other transforms, e.g., rotation or the Radon transform.

In [1]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from glob import glob

In [2]:
WORKING_PATH = "../../../../output/build-simple-model/"
FILE_LIST = glob(WORKING_PATH + "images_*.npy")

In [3]:
# run the program through a single set of images for demonstration
for img_file in FILE_LIST:
    print(img_file)

../../../../output/build-simple-model\images_0001_0023.npy


In [4]:
    imgs_to_process = np.load(img_file).astype(np.float64) # load the image as numpy array in float64
    print("on image", img_file)
    print(imgs_to_process)

on image ../../../../output/build-simple-model\images_0001_0023.npy
[[[-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  ..., 
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]]

 [[-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  ..., 
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]]

 [[-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  ..., 
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -2048. -2048.]
  [-2048. -2048. -2048. ..., -2048. -

In [5]:
    for i in range(len(imgs_to_process)):
        print(i) # only 3 images in the numpy arrray (for the 3 layers)

0
1
2


In [6]:
    for i in range(len(imgs_to_process)):
        img = imgs_to_process[i] # this is the img (a two dimensional numpy array)
        #Standardize the pixel values
        mean = np.mean(img)
        std = np.std(img)
        img = img-mean
        img = img/std # to standardize values you subtract mean and divide by standard deviation
        # note: A standardized variable is a variable that has been rescaled to have a mean of zero and a standard deviation of one

In [7]:
    for i in range(len(imgs_to_process)):
        # Find the average pixel value near the lungs
        # to renormalize washed out images
        middle = img[100:400,100:400] # grabs the all pixels excluding pixels in a 100px border of the image 
        mean = np.mean(middle)  # new mean which is the mean of pixels in the middle
        max = np.max(img) # max values in the current standardized pixel spectrum
        min = np.min(img) # min values in the current standardized pixel spectrum
        # To improve threshold finding, I'm moving the 
        # underflow and overflow on the pixel spectrum
        img[img==max]=mean 
        img[img==min]=mean # take all max/min values, and set them to mean values for sake of thresholding